# fasttransform

> Transform is the main building block of data pipelines in fastai. And elsewhere if you want.

## Development Notes

**⚠️  Work in Progress notes:**

**Context:**

My previous implementation of fastcore.Transform was done by reimplementing
`fastcore.typedispatch.TypeDispatch` with an implementation based on Plum.

It worked, with some minor breaking changes in fastai. But it wasn't simple enough.

In this notebook I re-build Transform from scratch directly using Plum.
The benefit will be that `fastcore.dispatch` can be removed.
Users will know to use Plum directly for typedispatch and see Transform as a tool that builds on top of it.
But at the cost of some more breaking changes.


**Plan in this notebook:**

I start with focusing on the `encode(s)` method.
Right now I have two implementations. 

One simpler implementation, which ran into some complications with global state if classes get redefined.


One more complicated implementation, using metaclasses. Which addresses those issues.

I think now's a good time to get some input again. Do you think I'm on the right track?
If so I'll continue by adding more of the features mentioned in Transform, i.e.:

- decodes and setups support
- implement return type casting (see note at the bottom of this notebook about that)
- transforms on tuples
- the classes in fastcore.transform that inherit from Transform
- Pipeline

**Open questions:**

- keep the initialization with enc=(f1,f2,f3) ...?) etc.


## Pre-Release checklist

- [x] Init repo
- [x] Copy existing Transform implementation
- [x] Add Quantum tests
- [ ] Implement new Transform
- [ ] Implement new Pipeline
- [ ] Decide to remove pandas dev dependency (desired)
- [ ] See if can remove fastcore dependency (not desired per se)
- [ ] Add docs re plum and autoreload (see plum docs)
- [ ] pin plum to some greater than version

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install fasttransform in Development mode

```sh
# make sure fasttransform package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to fasttransform
$ nbdev_prepare
```

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/AnswerDotAI/fasttransform.git
```

or from [conda][conda]

```sh
$ conda install -c AnswerDotAI fasttransform
```

or from [pypi][pypi]


```sh
$ pip install fasttransform
```


[repo]: https://github.com/AnswerDotAI/fasttransform
[docs]: https://AnswerDotAI.github.io/fasttransform/
[pypi]: https://pypi.org/project/fasttransform/
[conda]: https://anaconda.org/AnswerDotAI/fasttransform

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/AnswerDotAI/fasttransform
[docs]: https://AnswerDotAI.github.io/fasttransform/
[pypi]: https://pypi.org/project/fasttransform/
[conda]: https://anaconda.org/AnswerDotAI/fasttransform

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2